```{contents}
```
## Typed State Schema

In LangGraph, a **Typed State Schema** defines the **formal structure and data contract** of the global shared state using Python type hints.
It enforces **correctness, safety, clarity, and scalability** across all nodes in the graph.

---

### **1. Motivation and Intuition**

As LangGraph systems grow, the shared state becomes the **backbone of coordination**.
Without a schema, the state turns into an unstructured dictionary that is:

* error-prone
* difficult to reason about
* hard to scale
* unsafe for teams

Typed State Schema transforms the state into a **well-defined interface**.

| Problem Without Schema | Solution With Schema |
| ---------------------- | -------------------- |
| Implicit contracts     | Explicit contracts   |
| Silent bugs            | Type errors          |
| Poor readability       | Self-documenting     |
| Hard refactoring       | Safe evolution       |

---

### **2. Conceptual Model**

```
┌─────────────────────────┐
│  Typed State Schema     │
│  (Formal Data Model)    │
└─────────────┬───────────┘
              ↓
     ┌──────────────────┐
     │     Node A       │
     └──────────────────┘
              ↓
     ┌──────────────────┐
     │     Node B       │
     └──────────────────┘
              ↓
     ┌──────────────────┐
     │     Node C       │
     └──────────────────┘
```

All nodes **must read and write** data that conforms to the schema.

---

### **3. How Typed State Schema Works**

The schema is defined using `TypedDict` (or `BaseModel`).

```python
from typing import TypedDict, List

class State(TypedDict):
    user_query: str
    plan: str
    results: List[str]
    done: bool
```

The schema is passed into the graph builder:

```python
builder = StateGraph(State)
```

Now **every node** must return a dictionary that is **compatible** with this schema.

---

### **4. Why This Matters**

| Property    | Benefit                         |
| ----------- | ------------------------------- |
| Correctness | Eliminates structural bugs      |
| Safety      | Prevents invalid state writes   |
| Readability | State becomes documentation     |
| Scalability | Teams can evolve systems safely |
| Debugging   | Errors surface immediately      |

---

### **5. Typed vs Untyped State**

| Feature         | Untyped Dict | Typed State Schema |
| --------------- | ------------ | ------------------ |
| Validation      | None         | Enforced           |
| Documentation   | Implicit     | Explicit           |
| Refactoring     | Dangerous    | Safe               |
| Error detection | Runtime only | Early              |
| Maintenance     | Difficult    | Predictable        |

---

### **6. Example: Schema-Driven Node Design**

```python
def planner(state: State):
    goal = state["user_query"]
    plan = create_plan(goal)
    return {"plan": plan}
```

If `plan` is not a string or missing, LangGraph raises an error.

---

### **7. Schema Evolution (Production Design)**

```python
class State(TypedDict):
    user_query: str
    plan: str
    results: List[str]
    done: bool
    confidence: float   # safely added field
```

Existing nodes remain valid while new nodes can use `confidence`.

---

### **8. Typed Schema with Checkpointing**

Only fields defined in the schema are:

* persisted
* checkpointed
* versioned
* replayed

This ensures **stable long-running workflows**.

---

### **9. Anti-Patterns to Avoid**

| Anti-Pattern                  | Why Bad           |
| ----------------------------- | ----------------- |
| Using raw `dict`              | No guarantees     |
| Changing field names casually | Breaks graph      |
| Overloading fields            | Hidden bugs       |
| Storing mixed types           | Unstable behavior |

---

### **10. Mental Model**

> **Typed State Schema = API contract for your graph**

Nodes are clients and servers communicating through this contract.

---

### **11. When to Use Typed State Schema**

Always — for:

* agent systems
* multi-step pipelines
* long-running processes
* team-based development
* production deployments

---

### **12. Key Design Principle**

> **The state schema is the single source of truth for your system’s behavior.**

Correctly designed schemas produce **robust, maintainable, enterprise-grade LangGraph systems**.

---

### **Demonstration**



In [1]:
from typing import TypedDict, List
from langgraph.graph import StateGraph, END

class State(TypedDict):
    user_query: str
    plan: str
    results: List[str]
    done: bool

def planner(state: State):
    return {"plan": f"Process: {state['user_query']}"}

def executor(state: State):
    return {"results": [state["plan"] + " → executed"], "done": True}

builder = StateGraph(State)

builder.add_node("planner", planner)
builder.add_node("executor", executor)

builder.set_entry_point("planner")
builder.add_edge("planner", "executor")
builder.add_edge("executor", END)

graph = builder.compile()

output = graph.invoke({
    "user_query": "Build data pipeline",
    "plan": "",
    "results": [],
    "done": False
})

print(output)


{'user_query': 'Build data pipeline', 'plan': 'Process: Build data pipeline', 'results': ['Process: Build data pipeline → executed'], 'done': True}


**Output**

```
{
 'user_query': 'Build data pipeline',
 'plan': 'Process: Build data pipeline',
 'results': ['Process: Build data pipeline → executed'],
 'done': True
}
```
